In [1]:
import csv
import praw
import datetime
from config import CLIENT_ID, CLIENT_SECRET, USERNAME, PASSWORD, USER_AGENT

# Initialize the Reddit API client. You need to create a token on Reddit to do this. see documentation.
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT,
)

print(reddit.read_only) # show whther or not i have the authenticated connection with the API

False


In [3]:
# Get your subreddit
subreddit = reddit.subreddit("covidvaccinated")

#Just doublechecking the info... 
print(subreddit.display_name)
# Output: redditdev
print(subreddit.title)
# Output: reddit development
print(subreddit.description)
# Output: a subreddit for discussion of ...

covidvaccinated
CovidVaccinated
Vaccines are coming ! Many types, from may manufacturers, at different times, for different people and in different places. Lets talk about vaccination and vaccines


In [4]:
# Get all submissions from the subreddit. NOTE: PRAW ONLY ALLOWS DATA FOR THE LAST 1K POSTS ON THE SUB. 
# YOU CANNOT GO BACK ANY FURTHER IN TIME. 
submissions = subreddit.new(limit=None) #gimme all

# Open the CSV file for writing
csv_file = open('covidvaccinated_data_praw.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Title', 'Submission ID', 'Post Body', 'Post Flair', 'Post Author Flair', 'Locked', 'Submission Score', 'Comment', 'Timestamp', 'Deleted', 'Post Author', 'Comment Author', 'Distinguished', 'Nest Level', 'Total Comments', 'Comment Score'])

def write_comments(comments, nest_level, total_comments):
    for comment in comments:
        # Comment deleted?
        deleted = (comment.author is None)

        # Comment author
        comment_author = str(comment.author) if comment.author else None

        # Is Distinguished and if so, how?
        distinguished = comment.distinguished if comment.distinguished else ''

        # Convert the timestamp to a human-readable format!
        timestamp = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')

        # All of the datapoints. Write submission title, submission ID, post body, post flair, post author flair,
        # locked status, submission score, comment, timestamp, deletion status,
        # post author, comment author, distinguished status, nest level, total comments,
        # and comment score to the CSV file
        csv_writer.writerow([submission.title, submission.id, post_body, post_flair, post_author_flair, locked, submission.score,
                             comment.body, timestamp, deleted, post_author, comment_author, distinguished,
                             nest_level, total_comments, comment.score])

        # Recursively write nested comments
        if len(comment.replies) > 0:
            write_comments(comment.replies, nest_level + 1, total_comments)

for submission in submissions:
    # Get the body of the post
    post_body = submission.selftext

    # Get the post flair
    post_flair = submission.link_flair_text if submission.link_flair_text else ''

    # Get the post author flair
    post_author_flair = submission.author_flair_text if submission.author_flair_text else ''

    # Locked?
    locked = submission.locked

    # post author
    post_author = str(submission.author) if submission.author else None

    # Score
    submission_score = submission.score

    # Get all comments for the submission
    submission.comments.replace_more(limit=None)
    comments = submission.comments.list()

    # Write top-level comments
    total_comments = len(comments)
    write_comments(comments, 0, total_comments)

# Close the CSV file
csv_file.close()
